<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/SoftVcDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Voice Changer (soft-vc)
---

This note is a demo version of Voice Changer for soft-vc. This demo is customized so as to  run on Colab.

The full version is an application that runs on Docker on a local PC.

In general, the official version can convert audio smoothly with less time lag.

Detailed usage instructions can be found in [this repository](https://github.com/w-okada/voice-changer).

# Check GPU
GPU is required for soft-vc. Confirm GPU is assigned. 

In [1]:
# (1) Confirm GPU 
!nvidia-smi

Sun Sep 18 22:18:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install and import modules 
Install required modules.


In [3]:
# (2-1) Install Modules
!pip install fastapi
!pip install uvicorn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 80 kB 10.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 3.8 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 


In [5]:
# (2-2) Import Modules
import torch

# Load models
soft-vc needs 3 models.

In [6]:
# (3) Load modules
hubert = torch.hub.load("bshall/hubert:main", "hubert_soft").cuda()
acoustic = torch.hub.load("bshall/acoustic-model:main", "hubert_soft").cuda()
hifigan = torch.hub.load("bshall/hifigan:main", "hifigan_hubert_soft").cuda()

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/bshall/hubert/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt" to /root/.cache/torch/hub/checkpoints/hubert-soft-0d54a1f4.pt


  0%|          | 0.00/361M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/bshall/acoustic-model/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bshall/acoustic-model/releases/download/v0.1/hubert-soft-0321fd7e.pt" to /root/.cache/torch/hub/checkpoints/hubert-soft-0321fd7e.

  0%|          | 0.00/71.8M [00:00<?, ?B/s]

Downloading: "https://github.com/bshall/hifigan/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/bshall/hifigan/releases/download/v0.1/hifigan-hubert-discrete-bbad3043.pt" to /root/.cache/torch/hub/checkpoints/hifigan-hubert-discrete-bbad3043.pt


  0%|          | 0.00/54.9M [00:00<?, ?B/s]

Removing weight norm...


# Clone repository and configure


In [7]:
# (4-1) Clone Repository
!git clone --depth 1 https://github.com/w-okada/voice-changer.git
%cd voice-changer/demo/

Cloning into 'voice-changer'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 101 (delta 12), reused 70 (delta 6), pack-reused 0
Receiving objects: 100% (101/101), 18.97 MiB | 21.06 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/voice-changer/demo


In [8]:
# (4-2) settle the config file
!cp ../template/setting_softvc_colab.json ../frontend/dist/assets/setting.json
!cat ../frontend/dist/assets/setting.json


{
    "app_title": "voice-changer[soft-vc]",
    "majar_mode": "colab",
    "voice_changer_server_url": "/test",
    "sample_rate": 48000,
    "buffer_size": 1024,
    "prefix_chunk_size": 60,
    "chunk_size": 60,
    "speaker_ids": [999, 107],
    "speaker_names": ["---", "user"],
    "src_id": 107,
    "dst_id": 999,
    "vf_enable": true,
    "voice_changer_mode": "realtime",
    "gpu": 0,
    "available_gpus": [0],
    "avatar": {
        "enable_avatar": true,
        "motion_capture_face": true,
        "motion_capture_upperbody": true,
        "lip_overwrite_with_voice": true,
        "avatar_url": "./assets/vrm/zundamon/zundamon.vrm",
        "backgournd_image_url": "./assets/images/bg_natural_sougen.jpg",
        "background_color": "#0000dd",
        "chroma_key": "#0000dd",
        "avatar_canvas_size": [1280, 720],
        "screen_canvas_size": [1280, 720]
    },
    "advance": {
        "avatar_draw_skip_rate": 3,
        "screen_draw_skip_rate": 3,
        "visualizer_dr

# Invoke server

Invoke server with below (5-1)cell. And wait for ready with the command in (5-2)cell.

Please wait for a while, then you can see the messsage.
`Application startup complete.`.

In [9]:
# (5-1) Invoke server
PORT=8092
get_ipython().system_raw(f'python3 SoftVcServerFastAPI.py {PORT} colab >foo 2>&1 &')

In [12]:
# (5-2) Check server status (Run with Ctrl+Ret)
!cat foo

Using cache found in /root/.cache/torch/hub/bshall_hubert_main
Using cache found in /root/.cache/torch/hub/bshall_acoustic-model_main
Using cache found in /root/.cache/torch/hub/bshall_hifigan_main
INFO:     Will watch for changes in these directories: ['/content/voice-changer/demo']
INFO:     Uvicorn running on http://0.0.0.0:8092 (Press CTRL+C to quit)
INFO:     Started reloader process [281] using StatReload
ENV: colab
Removing weight norm...
Using cache found in /root/.cache/torch/hub/bshall_hubert_main
Using cache found in /root/.cache/torch/hub/bshall_acoustic-model_main
Using cache found in /root/.cache/torch/hub/bshall_hifigan_main
Using cache found in /root/.cache/torch/hub/bshall_hubert_main
Using cache found in /root/.cache/torch/hub/bshall_acoustic-model_main
Using cache found in /root/.cache/torch/hub/bshall_hifigan_main
INFO:     Started server process [290]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


# Invoke Proxy
Starts a proxy to access the web server.

Click on the URL displayed to open it and the application will open in a separate tab.

Since it is a Colab, it will take some time to load (about 30 seconds).

In [13]:
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}front/")

https://ayh2m6j6xsd-496ff2e9c6d22116-8092-colab.googleusercontent.com/front/
